# 训练模型：
1. 处理数据
2. 搭建网络
3. 定义损失函数
4. 编写数据生成器 1）通过类方法定义 2）定义生成器函数
5. 设置学习率，batch-size，损失函数，编译模型
6. 设置监控函数
7. 开始训练

In [5]:
import keras
from keras.models import Model
from keras.layers import Input, merge, Conv2D, ZeroPadding2D, UpSampling2D, concatenate, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalAveragePooling2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import BatchNormalization, Dropout, Flatten, Lambda
from keras.layers.advanced_activations import ELU, LeakyReLU
from keras.optimizers import Adam, RMSprop, SGD
from keras.regularizers import l2
from keras.layers.noise import GaussianDropout

ImportError: 
Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
If you're working with a numpy git repo, try `git clean -xdf` (removes all
files not under version control).  Otherwise reinstall numpy.

Original error was: cannot import name 'multiarray' from 'numpy.core' (D:\Anaconda3\lib\site-packages\numpy\core\__init__.py)


## 2. 搭建网络

In [29]:
def U_Net(input_shape=(512,512,3), num_class=2):

    nb_filter = [32,64,128,256,512]
    global bn_axis
    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
        img_input = Input(shape=input_shape, name='main_input')
    else:
        bn_axis = 1
        img_input = Input(shape=input_shape, name='main_input')
    
    conv1, conv2, conv3, conv4, conv5 = encoding(img_input, nb_filter)
    
    feature = decoding(conv1, conv2, conv3, conv4, conv5, nb_filter, bn_axis)
    
    unet_output = Conv2D(num_class, (1, 1), activation='sigmoid', name='output', 
                         kernel_initializer = 'he_normal', padding='same')(feature)

    model = Model(input=img_input, output=unet_output)

    return model


In [33]:
def encoding(img_input, nb_filter):
    conv1 = standard_conv(img_input, stage='1e', nb_filter=nb_filter[0])
    pool1 = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(conv1)

    conv2 = standard_conv(pool1, stage='2e', nb_filter=nb_filter[1])
    pool2 = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(conv2)

    conv3 = standard_conv(pool2, stage='3e', nb_filter=nb_filter[2])
    pool3 = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(conv3)

    conv4 = standard_conv(pool3, stage='4e', nb_filter=nb_filter[3])
    pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(conv4)

    conv5 = standard_conv(pool4, stage='5e', nb_filter=nb_filter[4])
    
    return conv1, conv2, conv3, conv4, conv5

def decoding(conv1, conv2, conv3, conv4, conv5, nb_filter, bn_axis):
    up4 = Conv2DTranspose(nb_filter[3], (2, 2), strides=(2, 2), name='up4', padding='same')(conv5)
    conv4 = concatenate([up4, conv4], name='merge4', axis=bn_axis)
    conv4 = standard_conv(conv4, stage='4d', nb_filter=nb_filter[3])

    up3 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up3', padding='same')(conv4)
    conv3 = concatenate([up3, conv3], name='merge3', axis=bn_axis)
    conv3 = standard_conv(conv3, stage='3d', nb_filter=nb_filter[2])

    up2 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up2', padding='same')(conv3)
    conv2 = concatenate([up2, conv2], name='merge2', axis=bn_axis)
    conv2 = standard_conv(conv2, stage='2d', nb_filter=nb_filter[1])

    up1 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up1', padding='same')(conv2)
    conv1 = concatenate([up1, conv1], name='merge1', axis=bn_axis)
    conv1 = standard_conv(conv1, stage='1d', nb_filter=nb_filter[0])
    
    return conv1

In [37]:
def standard_conv(input_tensor, stage, nb_filter, kernel_size=3,activation='relu'):

    x1 = Conv2D(nb_filter, (kernel_size, kernel_size), name='conv'+stage+'_1', 
                kernel_initializer = 'he_normal', padding='same')(input_tensor)
    layer = BatchNormalization(axis=3, name='bn'+stage+'_1')(x1)
    layer = Activation('relu', name='act'+stage+'_1')(layer)

    x2 = Conv2D(nb_filter, (kernel_size, kernel_size), name='conv'+stage+'_2', 
                kernel_initializer = 'he_normal', padding='same')(layer)
    layer = BatchNormalization(axis=3, name='bn'+stage+'_2')(x2)
    return Activation('relu', name='act'+stage+'_2')(layer)

## 3. 定义损失函数


In [19]:
import keras
from keras import backend as K

def dice_coefficient(y_true, y_pred,csmooth = 1.):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def bce_dice_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) - dice_coefficient(y_true, y_pred)

## 4. 编写生成器函数

In [6]:
import numpy as np
import copy

def generator(train_data, shuffle_index_list=True, batch_size=16):
    '''
    train_data: 数据集的路径
    shuffle_index_list：是否打乱数据顺序
    batch_size：
    '''
    while True:
        index_list = copy.copy(train_data)
        if shuffle_index_list:
            random.shuffle(index_list)
        x = []
        y = []

        for i in range(batch_size):
            index = index_list.pop()
            x_path = index + '/imaging.nii.gz'
            y_path = index + '/segmentation.nii.gz'
            
            image = get_data(x_path)
            label = get_data(y_path)

            x.append(image)
            y.append(label)

        x = np.array(x)
        y = np.array(y)
        yield x, y

## 5. 训练前的一系列设置

### 5.1 得到训练集和验证集的数据生成器

In [11]:
train_path = 'E:\训练集'
valid_path = 'E:\验证集'
batch_size = 16

training_generator = generator(train_path, shuffle_index_list=True, batch_size=batch_size)
validation_generator = generator(valid_path, shuffle_index_list=True, batch_size=batch_size)

# 每一轮在训练集和验证集上迭代的次数
n_train_steps = len(train_path)//batch_size
n_validation_steps = len(valid_path)//batch_size

### 5.2 编译模型

In [38]:
import os
from keras.models import load_model 

model_file = 'E:/图像分割/训练好的模型.h5'

custom_objects = {'bce_dice_loss': bce_dice_loss, 'dice_coefficient': dice_coefficient}
if os.path.exists(model_file):
    model = load_model(model_file, custom_objects=custom_objects)
else:
    initial_learning_rate = 0.001
    model = U_Net(input_shape=(512,512,3))
    model.compile(optimizer=Adam(lr=initial_learning_rate), loss=bce_dice_loss, metrics=[dice_coefficient])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("ma..., outputs=Tensor("ou...)`


In [1]:
model.summary(90)

NameError: name 'model' is not defined

### 5.3 设置回调函数，监控训练过程 

In [40]:
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger, ReduceLROnPlateau, EarlyStopping

model_file = 'E:/图像分割/训练好的模型.h5'
logging_file='training.log'
log_dir = '/log'
n_epochs = 50
early_stopping_patience=10
learning_rate_drop=0.0005
learning_rate_patience=5

tbCallBack = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True,)
tbCallBack.set_model(model)   
check_point = ModelCheckpoint(model_file, save_best_only=True)
csvlog = CSVLogger(logging_file, append=True)
reduce_lr = ReduceLROnPlateau(factor=learning_rate_drop, patience=learning_rate_patience, verbose=1)
early_stopping = EarlyStopping(verbose=1, patience=early_stopping_patience)
callbacks = [check_point, tbCallBack, reduce_lr, early_stopping]

## 6. 开始训练

In [ ]:
model.fit_generator(generator=training_generator,
                    steps_per_epoch=n_train_steps,
                    epochs=n_epochs,
                    validation_data=validation_generator,
                    validation_steps=n_validation_steps,
                    callbacks=callbacks

# 使用模型：
1. 载入训练好的模型
2. 输入数据，得到预测结果
3. 结果的后续处理

In [ ]:
from keras.models import load_model 

model_file = 'E:/图像分割/训练好的模型.h5'

custom_objects = {'bce_dice_loss': bce_dice_loss, 'dice_coefficient': dice_coefficient}
model = load_model(model_file, custom_objects=custom_objects)

In [ ]:
import numpy as np

img_path = 'E:/测试集/00000.png'
img = get_data(img_path)

# 输入shape： （1, 512, 512, 3）
y_pred = model.predict(img[np.newaxis,:,:,:])   # 输出shape： (1, 512, 512, 2)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(y_pred[:,:,20])

## 完